# Tensor Board

### 보기힘든 loss등의 결과 값들을 그래프를 통해 보는 것!

## 1단계
----------------------
### 어떤 함수를 log할지 정하기
#### w2_hist = tf.summary.histogram('weight2', w2)
#### cost_summ = tf.summary.scalar('cost', cost)

## 2단계
----------------------
### 하나하나 쓰지 않고 한 번에 쓰기 위해 merge하기
#### summary = tf.summary.merge_all()

## 3단계
----------------------
### 세션을 열고 summary를 어디위치에 기록할지 저장
#### writer = tf.summary.FileWriter('./logs')   => 파일 위치
#### writer.add_graph(sess.graph)

## 4단계
----------------------
### summary 텐서를 실행시키기
#### s, _ = sess.run([summary, optimizer], feed_dict=feed_dict)
#### writer.add_summary(s, global_step=global_step)             => 실제 기록하는 부분

## 5단계
----------------------
### 명령어로 실행하기
#### !tensorboard --logdir=./logs

------------------------
## 원격으로 launch하기
#### !ssh -L local_port:127.0.0.1:remote_port username@server.com
#### 예시 => !ssh -L 7007:127.0.0.1:6006    godkim@server.com
####         ! tensorboard --logdir=./logs/xor_logs

## Histogram
--------------------
### 하나의 값이 아닌 여러개의 값의 분포를 파악 가능

## Graph
---------------------
### tf.name_scope를 통해 압축시켜서 그래프를 깔끔하게 볼 수 있다

## Multiple Runs
---------------------
### 하이퍼파라미터만 다르게 해서 실행시키는 걸 비교할때
#### tensorboard --logdir=./logs/xor_logs
#### tensorboard --logdir=./logs/xor_logs_r0_01         => learning_rate만 0.01로 바꾼 것
#### tensorboard --logdir=./logs

## XOR tensorboard 만들기

In [1]:
import tensorflow as tf
import numpy as np

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, cost_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")


0 0.6999822
100 0.63682127
200 0.501098
300 0.42449814
400 0.39287806
500 0.37748414
600 0.36881733
700 0.36342162
800 0.35981405
900 0.3572715
1000 0.35540554
1100 0.35399193
1200 0.3528926
1300 0.35201937
1400 0.3513133
1500 0.3507336
1600 0.3502513
1700 0.34984535
1800 0.3495005
1900 0.3492049
2000 0.3489494
2100 0.34872693
2200 0.34853238
2300 0.34836084
2400 0.34820884
2500 0.34807396
2600 0.34795332
2700 0.34784526
2800 0.34774762
2900 0.3476599
3000 0.34758002
3100 0.34750777
3200 0.3474419
3300 0.3473817
3400 0.3473267
3500 0.34727615
3600 0.3472298
3700 0.34718686
3800 0.34714746
3900 0.3471111
4000 0.34707743
4100 0.34704608
4200 0.34701717
4300 0.34699023
4400 0.34696537
4500 0.346942
4600 0.34692016
4700 0.34689993
4800 0.34688082
4900 0.3468633
5000 0.3468467
5100 0.34683114
5200 0.34681642
5300 0.3468029
5400 0.3467899
5500 0.34677804
5600 0.3467667
5700 0.34675628
5800 0.34674624
5900 0.346737
6000 0.3467281
6100 0.34671974
6200 0.34671214
6300 0.3467046
6400 0.34669775
